# 任务2：ChatGLM API使用

In [13]:
import time
import jwt
import requests
from itertools import combinations
import numpy as np
from tqdm import tqdm

# 实际KEY，过期时间
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

def ask_glm(question, nretry=5):
    if nretry == 0:
        return None

    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token("7bf001734ef2fd7f7a55bf51dadd7cbb.BMAsoKRDFTmTEPwj", 1000)
    }
    data = {
        "model": "glm-3-turbo",
        "p": 0.5,
        "messages": [{"role": "user", "content": question}]
    }
    try:
        response = requests.post(url, headers=headers, json=data, timeout=10)
        return response.json()
    except:
        return ask_glm(question, nretry-1)

In [16]:
def regex_agent(question):
    prompt_template = '''你是一个专业的python的工程师，擅长编写各种的正则表达式。将下面的要求转换为正则匹配表达式，只需要输出表达式，不要有其他的输出。
{0}
    '''.format(question)

    return ask_glm(prompt_template)['choices'][0]['message']['content']

In [18]:
print(regex_agent("识别首字母大写单词的正则"))

```
\b[A-Z][a-z]*\b
```


In [19]:
print(regex_agent("识别首字母大写且字符个数小于10的正则"))

`^[A-Z][a-zA-Z]{0,9}$`


In [20]:
print(regex_agent("识别单词末尾为标点符号的正则"))

\b\w+[.,;!?]$


# 任务3：数据库内容解析

In [22]:
'''数据库解析'''
from typing import Union
import traceback
from sqlalchemy import create_engine, inspect, func, select, Table, MetaData
import pandas as pd

class DBParser:
    '''DBParser'''
    def __init__(self, db_url:str) -> None:
        '''初始化
        db_url: 数据库链接地址
        '''

        # 判断数据库类型
        if 'sqlite' in db_url:
            self.db_type = 'sqlite'
        elif 'mysql' in db_url:
            self.db_type = 'mysql'

        # 链接数据库
        self.engine = create_engine(db_url, echo=False)
        self.conn = self.engine.connect()
        self.db_url = db_url

        # 查看表明
        self.inspector = inspect(self.engine)
        self.table_names = self.inspector.get_table_names()

        self._table_fields = {} # 数据表字段
        self.foreign_keys = [] # 数据库外键
        self._table_sample = {} # 数据表样例

        # 依次对每张表的字段进行统计
        for table_name in self.table_names:
            print("Table ->", table_name)
            self._table_fields[table_name] = {}

            # 累计外键
            self.foreign_keys += [
                {
                    'constrained_table': table_name,
                    'constrained_columns': x['constrained_columns'],
                    'referred_table': x['referred_table'],
                    'referred_columns': x['referred_columns'],
                } for x in self.inspector.get_foreign_keys(table_name)
            ]

            # 获取当前表的字段信息
            table_instance = Table(table_name, MetaData(), autoload=True, autoload_with=self.engine)
            table_columns = self.inspector.get_columns(table_name)
            self._table_fields[table_name] = {x['name']:x for x in table_columns}

            # 对当前字段进行统计
            for column_meta in table_columns:
                # 获取当前字段
                column_instance = getattr(table_instance.columns, column_meta['name'])

                # 统计unique
                query = select(func.count(func.distinct(column_instance)))
                distinct_count = self.conn.execute(query).fetchone()[0]
                self._table_fields[table_name][column_meta['name']]['distinct'] = distinct_count

                # 统计most frequency value
                field_type = self._table_fields[table_name][column_meta['name']]['type']
                field_type = str(field_type)
                if 'text' in field_type.lower() or 'char' in field_type.lower():
                    query = (
                        select([column_instance, func.count().label('count')])
                        .group_by(column_instance)
                        .order_by(func.count().desc())
                        .limit(1)
                    )
                    top1_value = self.conn.execute(query).fetchone()[0]
                    self._table_fields[table_name][column_meta['name']]['mode'] = top1_value

                # 统计missing个数
                query = select(func.count()).filter(column_instance == None)
                nan_count = self.conn.execute(query).fetchone()[0]
                self._table_fields[table_name][column_meta['name']]['nan_count'] = nan_count

                # 统计max
                query = select(func.max(column_instance))
                max_value = self.conn.execute(query).fetchone()[0]
                self._table_fields[table_name][column_meta['name']]['max'] = max_value

                # 统计min
                query = select(func.min(column_instance))
                min_value = self.conn.execute(query).fetchone()[0]
                self._table_fields[table_name][column_meta['name']]['min'] = min_value

                # 任意取值
                query = select(column_instance).limit(10)
                random_value = self.conn.execute(query).all()
                random_value = [x[0] for x in random_value]
                random_value = [str(x) for x in random_value if x is not None]
                random_value = list(set(random_value))
                self._table_fields[table_name][column_meta['name']]['random'] = random_value[:3]

            # 获取表样例（第一行）
            query = select([table_instance])
            self._table_sample[table_name] = pd.DataFrame([self.conn.execute(query).fetchone()])
            self._table_sample[table_name].columns = [x['name'] for x in table_columns]


    def get_table_fields(self, table_name) -> pd.DataFrame:
        '''获取表字段信息'''
        return pd.DataFrame.from_dict(self._table_fields[table_name]).T

    def get_data_relations(self) -> pd.DataFrame:
        '''获取数据库链接信息（主键和外键）'''
        return pd.DataFrame(self.foreign_keys)

    def get_table_sample(self, table_name) -> pd.DataFrame:
        '''获取数据表样例'''
        return self._table_sample[table_name]

    def check_sql(self, sql) -> Union[bool, str]:
        '''检查sql是否合理

        参数
            sql: 待执行句子

        返回: 是否可以运行 报错信息
        '''
        try:
            self.engine.execute(sql)
            return True, 'ok'
        except:
            err_msg = traceback.format_exc()
            return False, err_msg

    def execute_sql(self, sql) -> bool:
        '''运行SQL'''
        result = self.engine.execute(sql)
        return list(result)

In [23]:
parser = DBParser('sqlite:///./bs_challenge_financial_14b_dataset/dataset/博金杯比赛数据.db')

Table -> A股公司行业划分表
Table -> A股票日行情表
Table -> 基金份额持有人结构
Table -> 基金债券持仓明细
Table -> 基金可转债持仓明细
Table -> 基金基本信息
Table -> 基金日行情表
Table -> 基金股票持仓明细
Table -> 基金规模变动表
Table -> 港股票日行情表


In [24]:
parser.table_names

['A股公司行业划分表',
 'A股票日行情表',
 '基金份额持有人结构',
 '基金债券持仓明细',
 '基金可转债持仓明细',
 '基金基本信息',
 '基金日行情表',
 '基金股票持仓明细',
 '基金规模变动表',
 '港股票日行情表']

In [25]:
parser.get_table_fields("A股公司行业划分表")

,name,type,nullable,default,autoincrement,primary_key,distinct,mode,nan_count,max,min,random
股票代码,股票代码,TEXT,True,None,auto,0,5205,990018,0,990018,000001,"[002549, 000065, 002561]"
交易日期,交易日期,TEXT,True,None,auto,0,1096,20211231,0,20211231,20190101,"[20190120, 20190121, 20190125]"
行业划分标准,行业划分标准,TEXT,True,None,auto,0,2,申万行业分类,0,申万行业分类,中信行业分类,[中信行业分类]
一级行业名称,一级行业名称,TEXT,True,None,auto,0,45,电子,309830,餐饮旅游,交通运输,"[电力设备及新能源, 交通运输, 建材]"
二级行业名称,二级行业名称,TEXT,True,None,auto,0,221,None,1974149,黑色家电Ⅱ,IT服务,"[一般零售, 建筑施工Ⅱ, 公交物流]"


In [45]:
def sql_agent(table_name, table_info, question):
    prompt_template = '''你是一个sql专家，基于已有的表格信息，请将下面的问题转换为sql查询语句。直接输出sql，不要输出其他内容。
表名称：{0}

表格信息：
{1}

待查询问题：{2}
'''.format(table_name, table_info, question)

    return ask_glm(prompt_template)['choices'][0]['message']['content']

In [46]:
sql_agent(
    "A股公司行业划分表",
    parser.get_table_fields("A股公司行业划分表").to_markdown(),
    '查询下总共有多少个股票'
)

'```sql\nSELECT COUNT(DISTINCT 股票代码) FROM A股公司行业划分表;\n```'

In [48]:
sql = sql_agent(
    "A股公司行业划分表",
    parser.get_table_fields("A股公司行业划分表").to_markdown(),
    '查询下总共有多少个股票'
)
sql = sql.replace('```sql', '').replace('```', '').strip()

In [49]:
parser.execute_sql(sql)

[(5205,)]

# 任务4：文本索引与答案检索

In [50]:
parser.table_names

['A股公司行业划分表',
 'A股票日行情表',
 '基金份额持有人结构',
 '基金债券持仓明细',
 '基金可转债持仓明细',
 '基金基本信息',
 '基金日行情表',
 '基金股票持仓明细',
 '基金规模变动表',
 '港股票日行情表']

In [79]:
import pdfplumber  # 导入pdfplumber模块，用于处理PDF文件
import glob
import random
import json
import numpy as np

# !pip install rank_bm25
from rank_bm25 import BM25Okapi

In [67]:
prompt_for_company_name = '''你是一个专业的识别公司名称的文本处理专家，请识别下面文档中的公司名称。直接输出公司的中文名称，不要有其他输出。
{0}
'''

In [78]:
pdf_company_list = []
for pdf_path in glob.glob('./bs_challenge_financial_14b_dataset/pdf/*'):
    pdf = pdfplumber.open(pdf_path)
    for i in range(10):
        first_page_content = pdf.pages[i].extract_text()
        first_page_content = first_page_content.strip()
        if len(first_page_content) > 0:
            break
    company_name = ask_glm(prompt_for_company_name.format(first_page_content))['choices'][0]['message']['content']
    company_name = company_name.split(' ')[0].split('\n')[0]
    print(pdf_path, company_name)
    pdf_company_list.append([pdf_path, company_name])

./bs_challenge_financial_14b_dataset/pdf/2389de12d78fe1ca4fa24910e6b1573902098bc3.PDF 深圳市铁汉生态环境股份有限公司
./bs_challenge_financial_14b_dataset/pdf/50b2823371fe1699d260f67cadac3d38af0672e3.PDF 浙江富春江环保热电股份有限公司
./bs_challenge_financial_14b_dataset/pdf/b96b11328c7bdc32b63fd15b1d1f96759ed94add.PDF 烟台杰瑞石油服务集团股份有限公司
./bs_challenge_financial_14b_dataset/pdf/54c7b3ab01ad11d37835a4464c9e4d68dfe6306f.PDF 云南沃森生物技术股份有限公司
./bs_challenge_financial_14b_dataset/pdf/044635429bd83e329c5047010121044e07568feb.PDF 上海华铭智能终端设备股份有限公司
./bs_challenge_financial_14b_dataset/pdf/810d8681429537dbf20a437a6dbf08e34c1ece27.PDF 浙江百达精工股份有限公司
./bs_challenge_financial_14b_dataset/pdf/e0a89442ee14a193c6600b0092a43a95a2ddab88.PDF 华达汽车科技股份有限公司
./bs_challenge_financial_14b_dataset/pdf/bec898d3079074d88f8bdb34d7e07f072cfca695.PDF 中国神华能源股份有限公司
./bs_challenge_financial_14b_dataset/pdf/398c8e64f18a13e695b5956122ef2f6a6fd3b274.PDF 上海真兰仪表科技股份有限公司
./bs_challenge_financial_14b_dataset/pdf/f7a306fea164e32f1df88b7beb8ebba8f65dd7a1.PDF 苏州海陆重

In [89]:
questions = json.load(open('博金杯金融问答_query.json'))
questions[0]

{'question': '在2019年1月15日，根据中信行业分类标准，能否帮我查询一下有多少家A股公司的股票代码是000065？相关的数据可以在A股公司行业划分表里找到，谢谢。',
 'answer': '',
 'reference': ''}

In [90]:
reference_words = [list(x) for x in parser.table_names]
reference_words += [list(x[1]) for x in pdf_company_list]
reference_path = parser.table_names + [x[0].split('/')[-1] for x in pdf_company_list]
bm25 = BM25Okapi(reference_words)

In [91]:
for query_idx in range(len(questions)):
    doc_scores = bm25.get_scores(list(questions[query_idx]["question"]))
    max_score_page_idx = doc_scores.argsort()[::-1][0]
    questions[query_idx]['reference'] = reference_path[max_score_page_idx]

In [94]:
with open('submit_reference_bm25.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)